In [1]:
from mainnet_launch.autopool_diagnostics.compute_rebalance_cost import (
    compute_daily_rebalance_costs,
    AUTO_LRT, BAL_ETH, AUTO_ETH,
    AutopoolConstants,
)
import pandas as pd
import plotly.express as px

a, b = compute_daily_rebalance_costs(AUTO_ETH)
rebalance_events_df, transfer_events_df, safe_lp_token_value_at_block_before_rebalance = b

2024-11-12 17:45:59.905 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-12 17:45:59.909 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-12 17:45:59.913 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-12 17:45:59.915 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-12 17:45:59.916 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-12 17:45:59.918 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-12 17:45:59.919 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-12 17:45:59.920 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-

In [2]:
import plotly.io as pio
pio.templates.default = None

In [5]:
def compute_swap_in_rebalances_from_idle(
    rebalance_from_idle_df: pd.DataFrame, autopool: AutopoolConstants, transfer_events_df: pd.DataFrame
):
    # out means decrease
    # in means increase
    if (rebalance_from_idle_df["outDestinationVault"] != autopool.autopool_eth_addr).any():
        raise ValueError(
            "trying to compute rebalance cost on rebalance_from_idle_df when the outDestinationVault != autopool "
        )
    
    # get all the transfers to the inDestinationsVaults, the indestination Vaut
    transfers_to_vaults = (
        transfer_events_df[
            (transfer_events_df["hash"].isin(rebalance_from_idle_df["hash"]))
            & (transfer_events_df["to"].isin(rebalance_from_idle_df["inDestinationVault"]))
        ]
        
        .drop_duplicates()
        .copy()
    )

    rebalance_from_idle_with_lp_tokens_in = pd.merge(
        rebalance_from_idle_df, transfers_to_vaults[["value", "hash"]], on="hash"
    )
    rebalance_from_idle_with_lp_tokens_in["lp_tokens_added_to_vault"] = (
        rebalance_from_idle_with_lp_tokens_in[["value"]] / 1e18
    )
    rebalance_from_idle_with_lp_tokens_in["weth_removed_from_idle"] = rebalance_from_idle_with_lp_tokens_in[
        "params_amountOut"
    ]
    return rebalance_from_idle_with_lp_tokens_in



rebalance_from_idle_df = rebalance_events_df.copy()
rebalance_from_idle_with_lp_tokens_in = compute_swap_in_rebalances_from_idle(
    rebalance_from_idle_df, AUTO_ETH, transfer_events_df
)
safe_lp_token_value_at_block_before_rebalance["block_with_rebalance"] = (
    safe_lp_token_value_at_block_before_rebalance["block"] + 1
)

rebalance_from_idle_df = pd.merge(
    rebalance_from_idle_with_lp_tokens_in,
    safe_lp_token_value_at_block_before_rebalance,
    left_on="block",
    right_on="block_with_rebalance",
)

# row = rebalance_from_idle_df.to_dict(orient='records')[0]


def _compute_spot_value_added_to_vault(row:dict) -> float:
    return row['lp_tokens_added_to_vault'] * row[row['inDestinationVault']] # the pool itself

def _compute_min_lp_tokens_needed(row:dict) -> float:
    # the min amount of lp tokens needed to be sent to the vault or the transaction will revert
    return float(row['params'][2] /1e18)

rebalance_from_idle_df['min_lp_tokens'] = rebalance_from_idle_df.apply(_compute_min_lp_tokens_needed, axis=1)
rebalance_from_idle_df['percent_excess_lp_tokens'] = 100 *(rebalance_from_idle_df['lp_tokens_added_to_vault'] - rebalance_from_idle_df['min_lp_tokens'])/ rebalance_from_idle_df['min_lp_tokens']
rebalance_from_idle_df['validated_spot_value_of_lp_tokens_added_to_autopool'] = rebalance_from_idle_df.apply(_compute_spot_value_added_to_vault, axis=1)
rebalance_from_idle_df['validated_spot_value_of_idle_eth_removed_from_autopool'] = rebalance_from_idle_df['weth_removed_from_idle'] # spot value of weth is always 1

rebalance_from_idle_df['validated_spot_value_of_swap_cost'] = rebalance_from_idle_df['validated_spot_value_of_idle_eth_removed_from_autopool'] -  rebalance_from_idle_df['validated_spot_value_of_lp_tokens_added_to_autopool']

rebalance_from_idle_df['swap_cost_from_event'] = rebalance_from_idle_df['valueStats'].apply(lambda x: float(x[4]) /1e18)
rebalance_from_idle_df['slippage_from_event'] = rebalance_from_idle_df['valueStats'].apply(lambda x: float(x[5]) /1e18)
rebalance_from_idle_df['validated_spot_slippage'] = (rebalance_from_idle_df['validated_spot_value_of_swap_cost'] / rebalance_from_idle_df['lp_tokens_added_to_vault'].astype(float)
)
px.line(rebalance_from_idle_df[['validated_spot_slippage', 'slippage_from_event']], title='event vs validated spot slippage')

In [16]:
rebalance_from_idle_df[['validated_spot_slippage', 'slippage_from_event']].median()

validated_spot_slippage    0.000375
slippage_from_event        0.000763
dtype: float64

In [17]:
rebalance_from_idle_df[['swap_cost_from_event', 'validated_spot_value_of_swap_cost']].clip(lower=0).sum()

swap_cost_from_event                 13.549673
validated_spot_value_of_swap_cost     9.164009
dtype: float64

In [26]:
rebalance_from_idle_df[['swap_cost_from_event', 'validated_spot_value_of_swap_cost']].sum()

swap_cost_from_event                 13.549673
validated_spot_value_of_swap_cost    -4.702996
dtype: float64

In [28]:
px.line(rebalance_from_idle_df[['swap_cost_from_event', 'validated_spot_value_of_swap_cost']], title='clipped event vs validated spot porice swap cost')

In [27]:
px.line(rebalance_from_idle_df[['swap_cost_from_event', 'validated_spot_value_of_swap_cost']].clip(lower=0), title='clipped event vs validated spot porice swap cost')

In [19]:
(rebalance_from_idle_df['validated_spot_value_of_swap_cost'] < 0).value_counts()

validated_spot_value_of_swap_cost
False    71
True     14
Name: count, dtype: int64

In [20]:
# sometimes the swap cost can be negative, 

In [21]:
px.histogram(rebalance_from_idle_df[['validated_spot_value_of_swap_cost']])

In [22]:
px.histogram(rebalance_from_idle_df[['swap_cost_from_event']])

In [23]:
rebalance_events_df

,valueStats,params,outSummaryStats,inSummaryStats,swapOffsetPeriod,predictedAnnualizedGain,event,block,transaction_index,log_index,hash,outDestinationLpToken,outDestinationVault,inDestinationLpToken,inDestinationVault,params_amountIn,params_amountOut
0,"(1000052485843985083, 1000000000000000000, 999...","(0xE382BBd32C4E202185762eA433278f4ED9E6151E, 0...","(0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56, 0...","(0xE382BBd32C4E202185762eA433278f4ED9E6151E, 0...",50,10041640546119019522,RebalanceBetweenDestinations,20772483,3,32,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xE382BBd32C4E202185762eA433278f4ED9E6151E,99.961014,100.000000
1,"(1000022248487157020, 1000000000000000000, 314...","(0xE382BBd32C4E202185762eA433278f4ED9E6151E, 0...","(0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56, 0...","(0xE382BBd32C4E202185762eA433278f4ED9E6151E, 0...",50,27968246236303374082,RebalanceBetweenDestinations,20774967,1,29,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xE382BBd32C4E202185762eA433278f4ED9E6151E,314.380624,314.500371
2,"(1008925796222361711, 1000000000000000000, 145...","(0xd96E943098B2AE81155e98D7DC8BeaB34C539f01, 0...","(0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56, 0...","(0xd96E943098B2AE81155e98D7DC8BeaB34C539f01, 1...",50,12637803214942412382,RebalanceBetweenDestinations,20778643,1,35,0xaa013ce865573532459a55487fb809c3e93edeed05af...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,144.676019,146.034135
3,"(1009075692904456357, 1000000000000000000, 134...","(0xd96E943098B2AE81155e98D7DC8BeaB34C539f01, 0...","(0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56, 0...","(0xd96E943098B2AE81155e98D7DC8BeaB34C539f01, 1...",50,11777498352828309122,RebalanceBetweenDestinations,20781802,0,33,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,133.437132,134.705433
4,"(1000144557246550759, 1000000000000000000, 335...","(0xE382BBd32C4E202185762eA433278f4ED9E6151E, 0...","(0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56, 0...","(0xE382BBd32C4E202185762eA433278f4ED9E6151E, 0...",50,27351106284624006252,RebalanceBetweenDestinations,20784928,5,42,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xE382BBd32C4E202185762eA433278f4ED9E6151E,335.615197,335.831303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"(1014562853847345824, 1000000000000000000, 998...","(0xdfE3fA7027E84f59b266459C567278C79fe86f0C, 0...","(0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56, 0...","(0xdfE3fA7027E84f59b266459C567278C79fe86f0C, 1...",48,8874192721267204642,RebalanceBetweenDestinations,21147722,1,48,0x7a63573ce03acfad5b3fe5c3beb465588686189af38e...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xB91159aa527D4769CB9FAf3e4ADB760c7E8C8Ea7,0xdfE3fA7027E84f59b266459C567278C79fe86f0C,98.404608,100.000000
95,"(1010851817059025162, 1000000000000000000, 999...","(0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04, 0...","(0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56, 0...","(0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04, 1...",48,6843760913900842704,RebalanceBetweenDestinations,21152202,0,30,0x5115e7697f0c5b1e5cd4891f3c19548332e376636f69...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,98.863864,100.000000
96,"(1001765855338677058, 1000000

In [24]:
# # return RebalanceValueStats({
# #     inPrice: inPrice,
# #     outPrice: outPrice,
# #     inEthValue: inEthValue,
# #     outEthValue: outEthValue,
# #     swapCost: swapCost,
# #     slippage: slippage
# # });


# /// @param destinationIn The address / lp token of the destination vault that will increase
# /// @param tokenIn The address of the underlyer token that will be provided by the swapper
# /// @param amountIn The amount of the underlying LP tokens that will be received
# /// @param destinationOut The address of the destination vault that will decrease
# /// @param tokenOut The address of the underlyer token that will be received by the swapper
# /// @param amountOut The amount of the tokenOut that will be received by the swapper
# struct RebalanceParams {
#     address destinationIn;
#     address tokenIn;
#     uint256 amountIn;
#     address destinationOut;
#     address tokenOut;
#     uint256 amountOut;
# }

In [25]:
rebalance_from_idle_df.sort_values('rebalance_cost')[['hash', 'rebalance_cost']].values[0]

KeyError: 'rebalance_cost'

0xf06978787ae75d235cedd61c07b1e2a9037df86378f37a90eb683c87db8f98b2

This rebalance, my math thinks that

In [ ]:
rebalance_from_idle_df['valueStats_swap_cost'] =rebalance_from_idle_df['valueStats'].apply(lambda x: x[4]/1e18)
px.line(rebalance_from_idle_df[['rebalance_cost', 'valueStats_swap_cost']])

In [ ]:
dest_to_dest = lp_token_transfer_df[(lp_token_transfer_df["hash"] == tx_hash)].copy().drop_duplicates()
dest_to_dest

NameError: name 'lp_token_transfer_df' is not defined

In [ ]:
# 165151197070039380229 expect outDestinationLpToken to be down

today I am working more on fixing the overestimate the rebalancing cost on the dashboard. 

In [ ]:
# idle -> destination valueStats.amountOut == quantiy of eth that left
#

In [ ]:
rebalance_events_df.shape

In [ ]:
160795862717038788608 + 419684887152871954

In [ ]:
dest_to_dest[dest_to_dest["token_address"] == inDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["token_address"] == outDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["to"] == outDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["from"] == outDestinationVault]
# destination vault -> flash borrow solver 0x067cB94576aBBDB3527055f3c8CaAAc9668364AD
# is the quanikty t

In [ ]:
dest_to_dest["to"].value_counts()

In [ ]:
rebalance_events_df.values[-1]  # 73804358107532979104

In [ ]:
rebalance_events_df